In [150]:
#
#imports
#

from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [148]:
#
#defining useful functions
#

def greyscale_from_array(im, brightening_factor=1):
    #stack the three channels side by side
    rgb = np.hstack((im[::,::,0:1:],im[::,::,1:2:],im[::,::,2:3:]))

    #change each list of one integer to just that integer
    rgb_fixed = np.zeros((rgb.shape[0], rgb.shape[1]))
    for i in range(rgb.shape[0]):
        for j in range(rgb.shape[1]):
            rgb_fixed[i][j] = rgb[i][j][0]*brightening_factor

    #return the greyscale image
    return rgb_fixed

def load_images_to_dict(key="stack", folder="train"):
    #for switching between "train", "test", and "valid" folders
    path = (folder + "/" + folder)

    #makes a dictionary mapping species to all the files in that folder
    directories = {}
    for species_file in os.listdir(path):
        directories[species_file] = os.listdir(path  + "/" + species_file)

    #makes a dictionary mapping species to arrays for each image in that folder
    images = {}
    for directory in directories.keys():
        images[directory] = []
        for file in os.listdir(path + "/" + directory): 
            if key == "stack":
                array = np.array(Image.open(path + "/" + directory + "/" + file).resize((24,36)))
                images[directory].append(np.hstack((array[::,::,0],array[::,::,1],array[::,::,2])))
            else:
                images[directory].append(np.array(Image.open(path + "/" + directory + "/" + file).resize((24,36)))[::,::,key])

    #return that dictionary
    return images

def build_X_y_from_dict(image_dict):
    X = []
    y = []
    
    i=0
    for key, value in image_dict.items():
        for image in value:
            X.append(image)
            y.append(i)
        i += 1
    
    y = np.array(y)
    X = np.array(X)
    X = np.reshape(X, (X.shape[0], X.shape[1]*X.shape[2]))

    return X, y

def normalize_Xtest_Xtrain(X_test, X_train):
    normalization_vector = np.zeros(X_train.shape[1])
    std_vector = np.zeros(X_train.shape[1])


    
    for index in range(X_train.shape[1]):
        normalization_vector[index] = sum(X_train[::, index])
        std_vector[index] = np.std(X_train[::, index])
    normalization_vector /= X_train.shape[0]


    
    X_test = X_test.astype('int16')
    for index in range(X_test.shape[0]):
        X_test[index] = X_test[index] - normalization_vector
    
    X_train = X_train.astype('int16')
    for index in range(X_train.shape[0]):
        X_train[index] = X_train[index] - normalization_vector


    
    return X_test, X_train

In [149]:
#
#train and score the data
#
train_images = load_images_to_dict("stack")
X_train, y_train = build_X_y_from_dict(train_images)
test_images = load_images_to_dict("stack", "test")
X_test, y_test = build_X_y_from_dict(test_images)

#X_test, X_train = normalize_Xtest_Xtrain(X_test, X_train)

clf = LogisticRegression(random_state=0, max_iter=100).fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.8181818181818182


C:\Users\ebstr\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
